In [1]:
import random
import torch
import numpy as np
import gensim
import gensim.downloader as glove_api
import os
import io

from matplotlib import pyplot as pl
import pickle

from ZorkGym.text_utils.text_parser import BagOfWords, Word2Vec, TextParser, tokenizer
from agents.OMP_DDPG import OMPDDPG

In [2]:
task = 'full'

In [3]:
if torch.cuda.is_available() and False:
    device = torch.device('cuda')
    torch.backends.cudnn.enabled = False
else:
    device = torch.device('cpu')

In [4]:
def word2vec_padding(list_of_embeddings, length, embedding_length):
    zero_vec = np.zeros(embedding_length)
    for _ in range(length - len(list_of_embeddings)):
        list_of_embeddings.append(zero_vec)
    return list_of_embeddings[:length]


def word2vec_sum(list_of_embeddings, embedding_length):
    ret_value = np.zeros(embedding_length)
    for embedding in list_of_embeddings:
        ret_value += embedding
    return ret_value

class OneHotParser(TextParser):
    def __init__(self, vocabulary, type_func):
        """

        :param vocabulary: List of strings representing the vocabulary.
        :param type_func: Function which converts the output to the desired type, e.g. np.array.
        """
        self.vocab = vocabulary
        self.vocab_size = len(self.vocab)
        TextParser.__init__(self, type_func)

    def __call__(self, x):
        one_hot = np.zeros((len(x), self.vocab_size))  # +1 for out of vocabulary tokens.
        for idx, token_list in enumerate(x):
            sentence = ' '.join(token_list)
            vocab_idx = self.vocab.index(sentence)
            one_hot[idx, vocab_idx] = 1

        return self.convert_type(one_hot)

def load_list_from_file(file_path):
    with open(file_path) as file:
        content = file.readlines()
    ret = []
    for elem in content:
        clean_elem = elem.strip()
        if len(clean_elem) > 0:
            ret.append(clean_elem)
    return ret

In [5]:
with open(os.getcwd() + '/data/zork_walkthrough_full.txt', 'rb') as f:
    data = pickle.load(f)

raw_actions = data['actions']
raw_states = data['states']

# with open(os.getcwd() + '/data/zork_walkthrough_egg_troll.txt', 'rb') as f:
#     data = pickle.load(f)
    
# for action in data['actions']:
#     raw_actions.append(action)

# for state in data['states']:
#     raw_states.append(state)
    
actions = set()
for action in raw_actions:
    for token in tokenizer(action):
        actions.add(token)
actions = list(actions)

In [6]:
verbs = ['go', 'take', 'open', 'grab', 'run', 'walk', 'climb', 'kill', 'light', 'get']

#basic_actions = ['open', 'egg', 'east', 'west', 'north', 'south', 'go', 'up', 'down', 'look', 'take']
basic_actions = ['open', 'egg', 'north', 'climb', 'tree', 'take']

extended_actions = ['grab', 'run', 'climb', 'walk', 'go', 'south', 'east', 'west']

basic_objects = ['egg', 'door', 'tree', 'leaves', 'nest']

obj_ext1 = ['bag', 'bottle', 'rope', 'sword', 'lantern', 'knife', 'mat', 'mailbox',
            'rug', 'case', 'axe', 'diamond', 'leaflet', 'news', 'brick']
action_ext1 = ['enter', 'open the window', 'turn lamp on', 'move rug', 'open trap door', 'hit troll with sword']

random_words = ['bring', 'wait', 'test', 'heave', 'squat', 'garbage', 'you', 'no', 'year']

def create_actions():
#     words = list()
#     words.append('')
#     for action in actions:
#         tokens = tokenizer(action)
#         for token in tokens:
#             if token not in words:
#                 words.append(token)

#     sentences = list()
#     for i, word1 in enumerate(words):
#         for word2 in words[i + 1:]:
#             if word1 in verbs:
#                 sentences.append(word1 + ' ' + word2)
#             else:
#                 sentences.append(word2 + ' ' + word1)
                
    words = set()
#     for action in sentences:
#         for word in tokenizer(action):
#             words.add(word)

    for action in raw_actions:
        for token in tokenizer(action):
            words.add(token)

    action_vocabulary = {}
    for word in words:
        action_vocabulary[word] = word2vec_model[word]
    action_vocabulary[''] = [0 for _ in range(len(action_vocabulary['north']))]

    embedding_size = len(action_vocabulary['open'])
    
    return actions, action_vocabulary, embedding_size, words

In [7]:
word2vec_model = glove_api.load('glove-wiki-gigaword-50')
embedding_size = word2vec_model.vector_size
word2vec_parser = Word2Vec(type_func=lambda x: torch.FloatTensor(x).to(device).unsqueeze(0),
                           word2vec_model=word2vec_model,
                           return_func=lambda x: word2vec_padding(x, 65, embedding_size))

In [8]:
def test(additional_prints, test_iterations):
    total_reward = 0
    iteration = 0
    with torch.no_grad():
        while iteration < test_iterations:
            idx = 0
            try:
                obs = agent.env.reset()
                
                done = False

                full_state = torch.zeros((agent.history_size,
                                          2,
                                          agent.input_width,
                                          agent.input_length), dtype=torch.float32).to(agent.device)
                
                episode_reward = 0
                while not done:
                    obs = agent._parse_state(obs).view(2, agent.input_width, agent.input_length)
                    full_state[:agent.history_size - 1] = full_state[1:]
                    full_state[-1] = obs

                    if additional_prints:
                        print('state:')
                        agent.env.render()
                        #print(data['states'][idx])
                        #print(torch.norm(full_state-word2vec_parser(data['states'][idx])))
                    idx += 1
                    
                    agent_output = agent.network[0](full_state.unsqueeze(0))
                    action, text_command = agent._get_action_beam_search(agent_output, full_state, 0, False, number_of_neighbors, False)
                    if additional_prints:
                        print('action:')
                        #print(agent_output)
                        #print(action)
                        print(text_command)

                    obs, reward, done, has_won = agent.env.step(text_command)

                    episode_reward += reward

                if additional_prints:
                    agent.env.render()

                total_reward += agent.env.env.get_score()
                iteration += 1
            except EnvironmentError:
                print('There was some issue with the Zork test env.')

    return total_reward * 1.0 / test_iterations

# Default Agent

In [9]:
number_of_neighbors=20

actions, action_vocabulary, embedding_size, words = create_actions()

In [41]:
agent = OMPDDPG(actions=action_vocabulary,
                state_parser=word2vec_parser,
                embedding_size=embedding_size,
                input_length=embedding_size,
                input_width=65,
                history_size=12,
                model_type='CNN',
                device=device,
                pomdp_mode=False,
                loss_weighting=1.0,
                linear=False,
                improved_omp=False,
                task=task)

In [42]:
path = os.getcwd() + '/imitation_agent_' + task + '/16000/'
agent.network[0].load_state_dict(torch.load(path + '/actor'))
agent.network[1].load_state_dict(torch.load(path + '/critic'))

In [43]:
number_of_neighbors = 112
result = test(True, 1)

state:


West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.


[]
action:
 north
state:
North of House
You are facing the north side of a white house. There is no door here, and all the windows are boarded up. To the north a narrow
path winds through the trees.


[]
action:
 north
state:
Forest Path
This is a path winding through a dimly lit forest. The path heads north-south here. One particularly large tree with some low
branches stands at the edge of the path.


[]
action:
 up
state:
Up a Tree
You are about 10 feet above the ground nestled among some large branches. The nearest branch above you is above your reach.
Beside you on the branch is a small bird's nest.
In the bird's nest is a large egg encrusted with precious jewels, apparently scavenged by a childless songbird. The egg is
covered with fine gold inlay, and ornamented in lapis lazuli and mother-of-pearl. Unlike most eggs, this one is hinged and
cl

action:
kill troll with sword
state:
The quickness of your thrust knocks the troll back, stunned.
The troll slowly regains his feet.


['sword', 'nasty knife', 'rope', 'brass lantern', 'clove garlic', 'jewel-encrusted egg']
action:
kill troll with sword
state:
The troll takes a fatal blow and slumps to the floor dead.
Almost as soon as the troll breathes his last breath, a cloud of sinister black fog envelops him, and when the fog lifts, the
carcass has disappeared.
Your sword is no longer glowing.


['sword', 'nasty knife', 'rope', 'brass lantern', 'clove garlic', 'jewel-encrusted egg']
action:
drop  egg
state:
The Troll Room
This is a small room with passages to the east and south and a forbidding hole leading west. Bloodstains and deep scratches
(perhaps made by an axe) mar the walls.
There is a jewel-encrusted egg here.
There is a bloody axe here.


['sword', 'nasty knife', 'rope', 'brass lantern', 'clove garlic']
action:
 east
state:
East-West Passage
This is a narrow east-west pa

action:
 south
state:
Altar
This is the south end of a large temple. In front of you is what appears to be an altar. In one corner is a small hole in the
floor which leads into darkness. You probably could not get back up it.


['sceptre', 'brass bell', 'black book', 'pair candles', 'torch', 'nasty knife', 'brass lantern', 'clove garlic']
action:
 pray
state:
Forest
This is a forest, with trees in all directions. To the east, there appears to be sunlight.
You hear in the distance the chirping of a song bird.


['sceptre', 'brass bell', 'black book', 'pair candles', 'torch', 'nasty knife', 'brass lantern', 'clove garlic']
action:
 east
state:
Forest Path
This is a path winding through a dimly lit forest. The path heads north-south here. One particularly large tree with some low
branches stands at the edge of the path.


['sceptre', 'brass bell', 'black book', 'pair candles', 'torch', 'nasty knife', 'brass lantern', 'clove garlic']
action:
 south
state:
North of House
You are facing the 

action:
 south
state:
North of House
You are facing the north side of a white house. There is no door here, and all the windows are boarded up. To the north a narrow
path winds through the trees.


['gold coffin', 'brass lantern', 'torch']
action:
 east
state:
Behind House
You are behind the white house. A path leads into the forest to the east. In one corner of the house there is a small window
which is open.


['gold coffin', 'brass lantern', 'torch']
action:
 west
state:
Kitchen
You are in the kitchen of the white house. A table seems to have been used recently for the preparation of food. A passage leads
to the west and a dark staircase can be seen leading upward. A dark chimney leads down and to the east is a small window which
is open.
A bottle is sitting on the table.
The glass bottle contains:
  A quantity of water
There is a brown sack here.
The brown sack contains:
  A lunch


['gold coffin', 'brass lantern', 'torch']
action:
 west
state:
Living Room
You are in the living roo

action:
 north
state:
Dam Lobby
This room appears to have been the waiting room for groups touring the dam. There are open doorways here to the north and east
marked "Private", and there is a path leading south over the top of the dam.
Some guidebooks entitled "Flood Control Dam #3" are on the reception desk.
There is a matchbook whose cover says "Visit Beautiful FCD#3" here.


['platinum bar', 'painting', 'pair candles', 'brass bell', 'black book', 'brass lantern', 'torch']
action:
drop  painting
state:
Dam Lobby
This room appears to have been the waiting room for groups touring the dam. There are open doorways here to the north and east
marked "Private", and there is a path leading south over the top of the dam.
Some guidebooks entitled "Flood Control Dam #3" are on the reception desk.
A painting by a neglected genius is here.
There is a matchbook whose cover says "Visit Beautiful FCD#3" here.


['platinum bar', 'pair candles', 'brass bell', 'black book', 'brass lantern', 'torch']
ac

...

action:
get  screwdriver
state:
Living Room
You are in the living room. There is a doorway to the east. To the west is a cyclops-shaped opening in an old wooden door, above
which is some strange gothic lettering, a trophy case, and a rug lying beside an open trap door.
There is a rusty knife here.
There is a clove of garlic here.
Your collection of treasures consists of:
    A beautiful brass bauble
    A golden clockwork canary
    A jewel-encrusted egg
    A chalice
    A painting
    A beautiful jeweled scarab
    A large emerald
    A pot of gold
    A sceptre
    A crystal trident
    A trunk of jewels
    A leather bag of coins
    A platinum bar
    A crystal skull
    A gold coffin


['screwdriver', 'brass lantern', 'torch']
action:
get  garlic
state:
Living Room
You are in the living room. There is a doorway to the east. To the west is a cyclops-shaped opening in an old wooden door, above
which is some strange gothic lettering, a trophy case, and a rug lying beside an open tra

action:
 east
state:
Coal Mine
This is a non-descript part of a coal mine.


['pile coal', 'clove garlic', 'brass lantern']
action:
 south
state:
Coal Mine
This is a non-descript part of a coal mine.


['pile coal', 'clove garlic', 'brass lantern']
action:
 north
state:
Gas Room
This is a small room which smells strongly of coal gas. There is a short climb up some stairs and a narrow tunnel leading east.
There is a sapphire-encrusted bracelet here.


['pile coal', 'clove garlic', 'brass lantern']
action:
 up
state:
Smelly Room
This is a small non-descript room. However, from the direction of a small descending staircase a foul odor can be detected. To
the south is a narrow tunnel.


['pile coal', 'clove garlic', 'brass lantern']
action:
 south
state:
Shaft Room
This is a large room, in the middle of which is a small shaft descending through the floor into darkness below. To the west and
the north are exits from this room. Constructed over the top of the shaft is a metal framework to wh

action:
put torch in basket
state:
Drafty Room
This is a small drafty room in which is the bottom of a long shaft. To the south is a passageway and to the east a very narrow
passage. In the shaft can be seen a heavy iron chain.
At the end of the chain is a basket.
The basket contains:
  A torch (providing light)
  A huge diamond


['screwdriver']
action:
put screwdriver in basket
state:
Drafty Room
This is a small drafty room in which is the bottom of a long shaft. To the south is a passageway and to the east a very narrow
passage. In the shaft can be seen a heavy iron chain.
At the end of the chain is a basket.
The basket contains:
  A screwdriver
  A torch (providing light)
  A huge diamond


[]
action:
 east
state:
Timber Room
This is a long and narrow passage, which is cluttered with broken timbers. A wide passage comes from the east and turns at the
west end of the room into a very narrow passageway. From the west comes a strong draft.
There is a brass lantern (battery-powered) he

action:
put torch in case
state:
Living Room
You are in the living room. There is a doorway to the east. To the west is a cyclops-shaped opening in an old wooden door, above
which is some strange gothic lettering, a trophy case, and a rug lying beside an open trap door.
There is a rusty knife here.
Your collection of treasures consists of:
    A torch (providing light)
    A huge diamond
    A beautiful brass bauble
    A golden clockwork canary
    A jewel-encrusted egg
    A chalice
    A painting
    A beautiful jeweled scarab
    A large emerald
    A pot of gold
    A sceptre
    A crystal trident
    A trunk of jewels
    A leather bag of coins
    A platinum bar
    A crystal skull
    A gold coffin


['jade figurine', 'screwdriver', 'sapphire-encrusted bracelet', 'brass lantern']
action:
put bracelet in case
state:
Living Room
You are in the living room. There is a doorway to the east. To the west is a cyclops-shaped opening in an old wooden door, above
which is some strange go

action:
 east
state:
Behind House
You are behind the white house. A path leads into the forest to the east. In one corner of the house there is a small window
which is open.


['ancient map', 'screwdriver', 'brass lantern']
action:
 north
state:
North of House
You are facing the north side of a white house. There is no door here, and all the windows are boarded up. To the north a narrow
path winds through the trees.


['ancient map', 'screwdriver', 'brass lantern']
action:
 west
state:
West of House
You are standing in an open field west of a white house, with a boarded front door. A secret path leads southwest into the
forest.
There is a small mailbox here.


['ancient map', 'screwdriver', 'brass lantern']
action:
 southwest
state:
Stone Barrow
You are standing in front of a massive barrow of stone. In the east face is a huge stone door which is open. You cannot see into
the dark of the tomb.


['ancient map', 'screwdriver', 'brass lantern']
action:
 west
Stone Barrow
You are standing

In [44]:
print(result)

350.0


In [14]:
for state in data['states']:
    print(state)
    print()

[['west', 'of', 'house', 'you', 'are', 'standing', 'in', 'an', 'open', 'field', 'west', 'of', 'a', 'white', 'house', 'with', 'a', 'boarded', 'front', 'door', 'there', 'is', 'a', 'small', 'mailbox', 'here'], []]

[['north', 'of', 'house', 'you', 'are', 'facing', 'the', 'north', 'side', 'of', 'a', 'white', 'house', 'there', 'is', 'no', 'door', 'here', 'and', 'all', 'the', 'windows', 'are', 'boarded', 'up', 'to', 'the', 'north', 'a', 'narrow', 'path', 'winds', 'through', 'the', 'trees'], []]

[['forest', 'path', 'this', 'is', 'a', 'path', 'winding', 'through', 'a', 'dimly', 'lit', 'forest', 'the', 'path', 'heads', 'north', 'south', 'here', 'one', 'particularly', 'large', 'tree', 'with', 'some', 'low', 'branches', 'stands', 'at', 'the', 'edge', 'of', 'the', 'path'], []]

[['up', 'a', 'tree', 'you', 'are', 'about', 'feet', 'above', 'the', 'ground', 'nestled', 'among', 'some', 'large', 'branches', 'the', 'nearest', 'branch', 'above', 'you', 'is', 'above', 'your', 'reach', 'beside', 'you', 'o